In [ ]:
%matplotlib inline

# 2.2 Deep learning basics and mathematical principles
Deep learning is not as difficult as imagined, and even simpler than some traditional machine learning. The mathematical knowledge used does not need to be particularly advanced. This chapter will explain the basic theories in deep learning while implementing some simple theories through hands-on use of PyTorch. This chapter contains a lot of content, so only a brief introduction


## 2.2.1 Supervised learning and unsupervised learning
Supervised learning, unsupervised learning, semi-supervised learning, and reinforcement learning are four common machine learning methods we come into contact with daily:

-Supervised learning: Train to obtain an optimal model (this model belongs to a set of functions) through the existing training samples (ie, known data and its corresponding output). Best), and then use this model to map all inputs to corresponding outputs.
-Unsupervised learning: The difference between it and supervised learning is that we do not have any training samples in advance, and we need to directly model the data.
-Semi-supervised learning: Combine a large amount of unlabeled data and a small amount of labeled data in the training phase. Compared with the model using all label data, the training model using the training set can be more accurate during training.
-Reinforcement learning: We set a reward function to confirm whether we are getting closer and closer to the goal. Similar to how we train a pet, we will reward him if we do it right, and we will punish him if we do it wrong. To achieve our training purpose.

Here we only focus on supervised learning, because most of our courses later use supervised learning methods. The data input during training and verification contains both input x and output y corresponding to x, that is, learning data The correct answer has been given in advance.

## 2.2.2 Linear Regreesion
Linear regression is a statistical analysis method that uses regression analysis in mathematical statistics to determine the quantitative relationship between two or more variables. It is widely used. Its expression is y = w'x+e, where e is a normal distribution whose error follows a mean value of 0.

In regression analysis, only one independent variable and one dependent variable are included, and the relationship between the two can be approximated by a straight line. This kind of regression analysis is called unary linear regression analysis. If the regression analysis includes two or more independent variables, and the relationship between the dependent variable and the independent variable is linear, it is called multiple linear regression analysis.
Excerpt from [Baidu Encyclopedia](https://baike.baidu.com/item/linear regression/8190345)

simply put:
Linear regression has a mapping f for input x and output y, y=f(x), and the form of f is aX+b. Among them, a and b are two adjustable parameters. When we train, we train the two parameters a and b.

Let's use pyTorch code to do a detailed explanation

In [ ]:
# Quote
# Note, here we use a new library called seaborn. If the package is not found, please use pip install seaborn to install
import torch
from torch.nn import Linear, Module, MSELoss
from torch.optim import SGD
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
torch.__version__

Let's define a linear function. Here we use $y = 5x + 7$, where 5 and 7 are the parameters a and b mentioned above. Let’s use matplot to visualize this function first.

In [ ]:
x = np.linspace(0,20,500)
y = 5*x + 7
plt.plot(x,y)

Below I generate some random points as our training data

In [ ]:
x = np.random.rand(256)
noise = np.random.randn(256) / 4
y = x * 5 + 7 + noise
df = pd.DataFrame()
df['x'] = x
df['y'] = y

Show the data we generated on the graph

In [ ]:
sns.lmplot(x='x', y='y', data=df);

We randomly generated some points. Next, we will use PyTorch to build a linear model to fit them. This is the so-called training process. Since there is only one layer of linear model, we will use it directly.

In [ ]:
model=Linear(1, 1)

The parameter (1, 1) represents the number of input and output features (feature) is 1. The expression of the `Linear` model is $y=w \cdot x+b$, where $w$ represents weight, and $b$ represents Bias

Loss function we use the mean square loss function: `MSELoss`, this will be described in detail later

In [ ]:
criterion = MSELoss()

Optimizer We choose the most common optimization method `SGD`, which is to calculate the gradient of `mini-batch` in each iteration, and then update the parameters with a learning rate of 0.01. The optimizer will also be introduced later in this chapter

In [ ]:
optim = SGD(model.parameters(), lr = 0.01)

Train 3000 times

In [ ]:
epochs = 3000

Prepare training data: The shape of `x_train`, `y_train` is (256, 1), which means that the size of `mini-batch` is 256, and the size of `feature` is 1. `astype('float32')` is for direct next step Convert to `torch.float`.

In [ ]:
x_train = x.reshape(-1, 1).astype('float32')
y_train = y.reshape(-1, 1).astype('float32')

Started training

In [ ]:
for i in range(epochs):
    # Sort the input and output data, where the input and output must be the Tensor type of torch
    inputs = torch.from_numpy(x_train)
    labels = torch.from_numpy(y_train)
    #Use the model to predict
    outputs = model(inputs)
    #The gradient is set to 0, otherwise it will accumulate
    optim.zero_grad()
    # Calculate loss
    loss = criterion(outputs, labels)
    # Backpropagation
    loss.backward()
    # Use the optimizer default method to optimize
    optim.step()
    if (i%100==0):
        #Print the loss function every 100 times to see the effect
        print('epoch {}, loss {:1.4f}'.format(i,loss.data.item()))

The training is complete, let's see what the results of the training are. Use `model.parameters()` to extract model parameters. $w$, $b$ are the model parameters we need to train
The data we expect $w=5$, $b=7$ can be compared

In [ ]:
[w, b] = model.parameters()
print (w.item(),b.item())

Visualize our model again and see the data we trained. If you don’t like seaborn, you can use matplot directly

In [ ]:
predicted = model.forward(torch.from_numpy(x_train)).data.numpy()
plt.plot(x_train, y_train,'go', label ='data', alpha = 0.3)
plt.plot(x_train, predicted, label ='predicted', alpha = 1)
plt.legend()
plt.show()

The above is a simple example of using PyTorch to do linear regression. Below we will give a detailed introduction to the above content
## 2.2.3 Loss Function
The loss function is used to estimate the degree of inconsistency between the predicted value of the model (output in our example) and the true value (y_train in the example). It is a non-negative real valued function. The smaller the loss function, the smaller the model’s The better the robustness.
The process of our training model is to use the optimization algorithm of gradient descent through continuous iterative calculations to make the loss function smaller and smaller. The smaller the loss function is, the better the algorithm is in the sense.

Here is an important point: because PyTorch uses mini-batch to calculate, the calculated result of the loss function has been averaged over mini-batch

Common (built-in PyTorch) loss functions are as follows:
### nn.L1Loss:
Enter the absolute value of the difference between x and target y. The dimensions of x and y are required to be the same (can be a vector or matrix), and the resulting loss dimensions are also corresponding to the same

$ loss(x,y)=1/n\sum|x_i-y_i| $


### nn.NLLLoss:
Negative log-likelihood loss function for multiple classification

$ loss(x, class) = -x[class]$

If the weights parameter is passed in NLLLoss, the loss will be weighted, and the formula becomes

$ loss(x, class) = -weights[class] * x[class] $

### nn.MSELoss:
Mean square loss function, mean square error between input x and target y

$ loss(x,y)=1/n\sum(x_i-y_i)^2 $

### nn.CrossEntropyLoss:
The cross entropy loss function for multi-classification, LogSoftMax and NLLLoss are integrated into one class, and the nn.NLLLoss function will be called, which we can understand as CrossEntropyLoss()=log_softmax() + NLLLoss()


 $ \begin{aligned} loss(x, class) &= -\text{log}\frac{exp(x[class])}{\sum_j exp(x[j]))}\ &= -x[class ] + log(\sum_j exp(x[j])) \end{aligned} $

 Because NLLLoss is used, the weight parameter can also be passed in. At this time, the calculation formula of loss becomes:

 $ loss(x, class) = weights[class] * (-x[class] + log(\sum_j exp(x[j]))) $

 Therefore, this loss function is generally used in the case of multi-classification

### nn.BCELoss:
Calculate the binary cross entropy between x and y.

$ loss(o,t)=-\frac{1}{n}\sum_i(t[i]* log(o[i])+(1-t[i])* log(1-o[i] )) $

Similar to NLLLoss, you can also add weight parameters:

$ loss(o,t)=-\frac{1}{n}\sum_iweights[i]* (t[i]* log(o[i])+(1-t[i])* log(1- o[i])) $

When using, you need to add the Sigmoid function in front of the layer.

## 2.2.4 Gradient Descent
When introducing the loss function, we have already said that gradient descent is an optimization algorithm that makes the loss function smaller and smaller. When there is no model parameter to solve the machine learning algorithm, that is, the constrained optimization problem, the gradient descent (Gradient Descent) is the most One of the commonly used methods. So gradient descent is the core of what we currently call machine learning. After understanding its meaning, you also understand the meaning of machine learning algorithms.


### Gradient
In calculus, the partial derivative of the parameter of a multivariate function is obtained, and the partial derivative of each parameter obtained is written in the form of a vector, which is the gradient.
For example, the function f(x,y), which takes the partial derivatives of x and y respectively, the obtained gradient vector is (∂f/∂x, ∂f/∂y)T, referred to as grad f(x,y) or ▽f (x,y).

## 2.2.4 梯度下降
在介绍损失函数的时候我们已经说了，梯度下降是一个使损失函数越来越小的优化算法，在无求解机器学习算法的模型参数，即约束优化问题时，梯度下降（Gradient Descent）是最常采用的方法之一。所以梯度下降是我们目前所说的机器学习的核心，了解了它的含义，也就了解了机器学习算法的含义。


### 梯度
在微积分里面，对多元函数的参数求∂偏导数，把求得的各个参数的偏导数以向量的形式写出来，就是梯度。
例如函数f(x,y), 分别对x，y求偏导数，求得的梯度向量就是(∂f/∂x, ∂f/∂y)T，简称grad f(x,y)或者▽f(x,y)。

几何上讲，梯度就是函数变化增加最快的地方，沿着梯度向量的方向，更加容易找到函数的最大值。反过来说，沿着梯度向量相反的方向梯度减少最快，也就是更加容易找到函数的最小值。

我们需要最小化损失函数，可以通过梯度下降法来一步步的迭代求解，得到最小化的损失函数，和模型参数值。
### 梯度下降法直观解释
梯度下降法就好比下山，我们并不知道下山的路，于是决定走一步算一步，每走到一个位置的时候，求解当前位置的梯度，沿着梯度的负方向，也就是当前最陡峭的位置向下走一步，然后继续求解当前位置梯度，向这一步所在位置沿着最陡峭最易下山的位置走一步。这样一步步的走下去，一直走到觉得我们已经到了山脚。

如下图所示，（此图摘自百度百科）
![](1.png)

这样走下去，有可能我们不能走到山脚，而是到了某一个局部的山峰低处（局部最优解）。

这个问题在以前的机器学习中可能会遇到，因为机器学习中的特征比较少，所以导致很可能陷入到一个局部最优解中出不来，但是到了深度学习，动辄百万甚至上亿的特征，出现这种情况的概率几乎为0，所以我们可以不用考虑这个问题。

### Mini-batch的梯度下降法
对整个训练集进行梯度下降法的时候，我们必须处理整个训练数据集，然后才能进行一步梯度下降，即每一步梯度下降法需要对整个训练集进行一次处理，如果训练数据集很大的时候处理速度会很慢，而且也不可能一次的载入到内存或者显存中，所以我们会把大数据集分成小数据集，一部分一部分的训练，这个训练子集即称为Mini-batch。
在PyTorch中就是使用这种方法进行的训练，可以看看上一章中关于dataloader的介绍里面的batch_size就是我们一个Mini-batch的大小。

为了介绍的更简洁，使用 吴恩达老师的 [deeplearning.ai](https://www.deeplearning.ai/deep-learning-specialization/) 课程板书。


对于普通的梯度下降法，一个epoch只能进行一次梯度下降；而对于Mini-batch梯度下降法，一个epoch可以进行Mini-batch的个数次梯度下降。
![](2.png)
普通的batch梯度下降法和Mini-batch梯度下降法代价函数的变化趋势，如下图所示：
![](3.png)
- 如果训练样本的大小比较小时，能够一次性的读取到内存中，那我们就不需要使用Mini-batch，
- 如果训练样本的大小比较大时，一次读入不到内存或者现存中，那我们必须要使用 Mini-batch来分批的计算
- Mini-batch size的计算规则如下，在内存允许的最大情况下使用2的N次方个size
![](4.png)


`torch.optim`是一个实现了各种优化算法的库。大部分常用优化算法都有实现，我们直接调用即可。
### torch.optim.SGD
随机梯度下降算法，带有动量（momentum）的算法作为一个可选参数可以进行设置，样例如下：

In [ ]:
#lr参数为学习率，对于SGD来说一般选择0.1 0.01.0.001，如何设置会在后面实战的章节中详细说明
##如果设置了momentum，就是带有动量的SGD，可以不设置
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

### torch.optim.RMSprop
除了以上的带有动量Momentum梯度下降法外，RMSprop（root mean square prop）也是一种可以加快梯度下降的算法，利用RMSprop算法，可以减小某些维度梯度更新波动较大的情况，使其梯度下降的速度变得更快

In [ ]:
#我们的课程基本不会使用到RMSprop所以这里只给一个实例
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.01, alpha=0.99)

### torch.optim.Adam
Adam 优化算法的基本思想就是将 Momentum 和 RMSprop 结合起来形成的一种适用于不同深度学习结构的优化算法

In [ ]:
# 这里的lr，betas，还有eps都是用默认值即可，所以Adam是一个使用起来最简单的优化方法
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08)

## 2.2.5 方差/偏差
- 偏差度量了学习算法的期望预测与真实结果的偏离程序，即刻画了学习算法本身的拟合能力
- 方差度量了同样大小的训练集的变动所导致的学习性能的变化，即模型的泛化能力
![](5.png)

从图中我们可以看出
- 高偏差（high bias）的情况，一般称为欠拟合（underfitting），即我们的模型并没有很好的去适配现有的数据，拟合度不够。
- 高方差（high variance）的情况一般称作过拟合（overfitting），即模型对于训练数据拟合度太高了，失去了泛化的能力。

如何解决这两种情况呢？

欠拟合：
- 增加网络结构，如增加隐藏层数目；
- 训练更长时间；
- 寻找合适的网络架构，使用更大的NN结构；

过拟合 ：
- 使用更多的数据；
- 正则化（ regularization）；
- 寻找合适的网络结构；

例如我们上面的例子，可以计算出我们的偏差:

In [ ]:
print (5-w.data.item(),7-b.data.item())

## 2.2.6 正则化
利用正则化来解决High variance 的问题，正则化是在 Cost function 中加入一项正则化项，惩罚模型的复杂度，这里我们简单的介绍一下正则化的概念

### L1正则化
损失函数基础上加上权重参数的绝对值

$ L=E_{in}+\lambda{\sum_j} \left|w_j\right|$

### L2正则化
损失函数基础上加上权重参数的平方和

$ L=E_{in}+\lambda{\sum_j} w^2_j$

需要说明的是：l1 相比于 l2 会更容易获得稀疏解

[知乎](https://www.zhihu.com/question/37096933/answer/70507353)

In [ ]:
## 2.2.4 Gradient Descent
When introducing the loss function, we have already said that gradient descent is an optimization algorithm that makes the loss function smaller and smaller. When there is no model parameter to solve the machine learning algorithm, that is, the constrained optimization problem, the gradient descent (Gradient Descent) is the most One of the commonly used methods. So gradient descent is the core of what we currently call machine learning. After understanding its meaning, you also understand the meaning of machine learning algorithms.


### Gradient
In calculus, the partial derivative of the parameter of a multivariate function is obtained, and the partial derivative of each parameter obtained is written in the form of a vector, which is the gradient.
For example, the function f(x,y), which takes the partial derivatives of x and y respectively, the obtained gradient vector is (∂f/∂x, ∂f/∂y)T, referred to as grad f(x,y) or ▽f (x,y).

Geometrically speaking, the gradient is where the function change increases the fastest. Along the direction of the gradient vector, it is easier to find the maximum value of the function. Conversely, the gradient decreases the fastest along the opposite direction of the gradient vector, which means it is easier to find the minimum value of the function.

We need to minimize the loss function, which can be solved step by step by the gradient descent method to obtain the minimized loss function and model parameter values.
### Intuitive explanation of gradient descent method
The gradient descent method is like going down the mountain. We don’t know the way down the mountain, so we decided to take one step. When we reach a position, we solve the gradient of the current position along the negative direction of the gradient, which is the current steepest position. Go down one step, and then continue to solve the current position gradient, and take a step along the steepest and easiest downhill position to the position of this step. Going this way step by step, until we feel that we have reached the foot of the mountain.

As shown in the figure below, (this figure is taken from Baidu Encyclopedia)
![](1.png)

If we continue this way, it is possible that we cannot go to the foot of the mountain, but to a certain local peak (local optimal solution).

This problem may be encountered in previous machine learning, because there are fewer features in machine learning, so it is likely to fall into a local optimal solution, but in deep learning, there are millions or even hundreds of millions of Feature, the probability of this situation is almost 0, so we don’t need to consider this problem.

### Mini-batch gradient descent method
When performing gradient descent on the entire training set, we must process the entire training data set before performing one-step gradient descent, that is, each step of the gradient descent method needs to process the entire training set once, if the training data set is large, it will be processed The speed will be very slow, and it is impossible to load into the memory or video memory at once, so we will divide the large data set into small data sets, part of the training, this training subset is called Mini-batch.
In PyTorch, this method is used for training. You can see the batch_size in the introduction to the dataloader in the previous chapter is the size of our Mini-batch.

In order to make the introduction more concise, use the [deeplearning.ai](https://www.deeplearning.ai/deep-learning-specialization/) course blackboard written by Wu Enda.


For the ordinary gradient descent method, one epoch can only perform gradient descent; and for the Mini-batch gradient descent method, one epoch can perform multiple gradient descents of Mini-batch.
![](2.png)
The change trend of the cost function of the ordinary batch gradient descent method and the Mini-batch gradient descent method is shown in the following figure:
![](3.png)
-If the size of the training sample is relatively small and can be read into the memory at once, then we don’t need to use Mini-batch,
-If the size of the training sample is relatively large, it cannot be read into the memory at a time or is currently stored, then we must use Mini-batch to calculate in batches
-The calculation rules of Mini-batch size are as follows, use 2 to the Nth power of the maximum size allowed by the memory
![](4.png)


`torch.optim` is a library that implements various optimization algorithms. Most commonly used optimization algorithms have been implemented, and we can call them directly.
### torch.optim.SGD
Stochastic gradient descent algorithm, the algorithm with momentum (momentum) can be set as an optional parameter, for example:

In [ ]:
#lr parameter is the learning rate. For SGD, 0.1 0.01.0.001 is generally selected. How to set it will be explained in detail in the actual chapter
##If momentum is set, it is SGD with momentum, you don’t need to set it
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

### torch.optim.RMSprop
In addition to the above Momentum gradient descent method with momentum, RMSprop (root mean square prop) is also an algorithm that can speed up the gradient descent. Using the RMSprop algorithm, you can reduce the situation of large dimensional gradient update fluctuations and make the gradient The rate of decline becomes faster

In [ ]:
#Our course basically does not use RMSprop, so here is only an example
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.01, alpha=0.99)

### torch.optim.Adam
The basic idea of ​​the Adam optimization algorithm is to combine Momentum and RMSprop to form an optimization algorithm suitable for different deep learning structures

In [ ]:
# Here lr, betas, and eps are all the default values, so Adam is the simplest optimization method to use
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08)

## 2.2.5 variance/bias
-Deviation measures the deviation of the expected prediction of the learning algorithm from the true result, and immediately describes the fitting ability of the learning algorithm itself
-Variance measures the changes in learning performance caused by changes in the training set of the same size, that is, the generalization ability of the model
![](5.png)

From the figure we can see
-The condition of high bias is generally called underfitting, that is, our model does not fit the existing data well, and the fit is not enough.
-The situation of high variance is generally called overfitting, that is, the model fits the training data too high and loses the ability to generalize.

How to solve these two situations?

Underfitting:
-Increase the network structure, such as increasing the number of hidden layers;
-Training longer;
-Find a suitable network architecture and use a larger NN structure;

Overfitting:
-Use more data;
-Regularization (regularization);
-Find a suitable network structure;

For example, in our example above, we can calculate our deviation:

In [ ]:
print (5-w.data.item(),7-b.data.item())

## 2.2.6 Regularization
Use regularization to solve the problem of high variance. Regularization is to add a regularization term to the Cost function to punish the complexity of the model. Here we briefly introduce the concept of regularization

### L1 regularization
Add the absolute value of the weight parameter to the loss function

$ L=E_{in}+\lambda{\sum_j} \left|w_j\right|$

### L2 regularization
Add the sum of squares of the weight parameters to the loss function

$ L=E_{in}+\lambda{\sum_j} w^2_j$

It should be noted that: l1 is easier to obtain sparse solutions than l2

[Knowledge](https://www.zhihu.com/question/37096933/answer/70507353)